In [10]:
import pandas as pd
import os
import re
from pandarize.frame import Pandarizer

#### Tex file parsing

In [2]:
dirpath = 'section2'
lst = []

for _, _, files in os.walk(dirpath):
    for fn in files:
        if os.path.splitext(fn)[1] == '.tex':
            filepath = os.path.join(dirpath, fn)
            print(filepath)
            with open(filepath, 'r') as f:
                for l in f.readlines():
                    if l[0] != '%':
                        lst.append(l.strip())

section2\0_abstract.tex
section2\1_intro.tex
section2\2_NI.tex
section2\3_AI.tex
section2\4_disc.tex
section2\diversity.tex


In [4]:
output = []

for line in lst:
    tmp = re.findall('cit(e|et)\{(.*?)\}', line)
    if tmp:
        for t in tmp:
            output += t[1].split(',')

final = []
for i in output:
    final += [i.strip()]

print(f'Length of the raw list: {len(final)}')

df = pd.DataFrame(final, columns=['text']).drop_duplicates('text')

print(f'Length after removing duplicates: {df.shape[0]}')

Length of the raw list: 104
Length after removing duplicates: 92


In [5]:
df.to_excel('output.xlsx', index=False)

#### Bib file parsing

In [34]:
bibpath = 'section2/references.bib'
bib_out = []
keep = False

with open(bibpath, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line[0] == '@':
            tmp = line
            keep = True
            continue
        elif line[0] == '}':
            tmp += line
            keep = False
            bib_out.append(tmp)
        
        if keep:
            tmp += line

In [36]:
bib_out[0]

'@article{Madry2018-gy,\n  title={Towards deep learning models resistant to adversarial attacks},\n  author={Madry, Aleksander and Makelov, Aleksandar and Schmidt, Ludwig and Tsipras, Dimitris and Vladu, Adrian},\n  journal={arXiv preprint arXiv:1706.06083},\n  year={2017}\n}\n'

In [37]:
authors = df['text'].tolist()
bib_final = []

for author in authors:
    for bib in bib_out:
        if author in bib:
            bib_final.append(bib)

len(bib_final)

92

#### Reconstruct bib files

In [40]:
with open('output.bib', 'w') as f:
    for line in bib_final:
        f.writelines(line)
        f.writelines('\n')

In [38]:
bib_final[0]

'@book{kandel2000principles,\n  title={Principles of neural science},\n  author={Kandel, Eric R and Schwartz, James H and Jessell, Thomas M and Siegelbaum, Steven and Hudspeth, A James and Mack, Sarah and others},\n  volume={4},\n  year={2000},\n  publisher={McGraw-hill New York}\n}\n'